## Colab url
https://colab.research.google.com/drive/1R6nEezCr-MayFCobKtLLtqjCWQ3npObu?usp=sharing

# Store Sales Data Exploration

Exploratory Data Analysis for the Store Sales Time Series Forecasting dataset from Kaggle.

## Dataset Overview:
- **train.csv**: Sales data with store, item family, and dates
- **stores.csv**: Store metadata (city, state, type, cluster)
- **oil.csv**: Daily oil prices (economic indicator)
- **holidays_events.csv**: Holiday and event information
- **transactions.csv**: Number of transactions per store/date

## Analysis Goals:
- Understand data structure and quality
- Identify sales patterns and trends
- Explore seasonality and holidays impact
- Analyze store and product performance

In [ ]:
# Import bibliotek
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path

# Setup
warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

# Data path
DATA_PATH = Path('../data/raw')

print(f"Data files available: {list(DATA_PATH.glob('*.csv'))}")

## 1. Data Loading

In [ ]:
# Load datasets
train = pd.read_csv(DATA_PATH / 'train.csv', parse_dates=['date'])
stores = pd.read_csv(DATA_PATH / 'stores.csv')
oil = pd.read_csv(DATA_PATH / 'oil.csv', parse_dates=['date'])
holidays = pd.read_csv(DATA_PATH / 'holidays_events.csv', parse_dates=['date'])
transactions = pd.read_csv(DATA_PATH / 'transactions.csv', parse_dates=['date'])

print("Dataset shapes:")
for name, df in [('train', train), ('stores', stores), ('oil', oil), ('holidays', holidays), ('transactions', transactions)]:
    print(f"{name}: {df.shape}")

In [ ]:
print("Train data sample:")
print(train.head(50))
print("\nData types:")
print(train.dtypes)
print(f"\nDate range: {train.date.min()} to {train.date.max()}")
print(f"Unique stores: {train.store_nbr.nunique()}")
print(f"Product families: {train.family.nunique()}")

## 2. Base data

In [ ]:
# Data quality check
print("Missing values:")
for name, df in [('train', train), ('stores', stores), ('oil', oil), ('holidays', holidays), ('transactions', transactions)]:
    missing = df.isnull().sum().sum()
    print(f"{name}: {missing} ({missing/df.size*100:.1f}%)")

print("\nTrain dataset statistics:")
print(train.describe())

neg_sales = (train.sales < 0).sum()
zero_sales = (train.sales == 0).sum()
print(f"\nNegative sales records: {neg_sales} ({neg_sales/len(train)*100:.2f}%)")
print(f"Zero sales records: {zero_sales} ({zero_sales/len(train)*100:.2f}%)")

# Sales distribution analysis (excluding zero sales)
positive_sales = train[train.sales > 0]['sales']
print(f"\n=== SALES DISTRIBUTION ANALYSIS (Sales > 0) ===")
print(f"Positive sales records: {len(positive_sales):,}")
print(f"Sales range: ${positive_sales.min():.2f} - ${positive_sales.max():.2f}")
print(f"Mean sales: ${positive_sales.mean():.2f}")
print(f"Median sales: ${positive_sales.median():.2f}")
print(f"Sales std: ${positive_sales.std():.2f}")

# Sales percentiles
percentiles = [10, 25, 50, 75, 90, 95, 99]
print("\nSales percentiles:")
for p in percentiles:
    value = positive_sales.quantile(p/100)
    print(f"{p}th percentile: ${value:.2f}")

# Plot sales distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Raw sales distribution
axes[0,0].hist(positive_sales, bins=50, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Sales Distribution (Sales > 0)')
axes[0,0].set_xlabel('Sales ($)')
axes[0,0].set_ylabel('Frequency')

# Log scale distribution
axes[0,1].hist(positive_sales, bins=50, alpha=0.7, edgecolor='black')
axes[0,1].set_title('Sales Distribution (Log Scale, Sales > 0)')
axes[0,1].set_xlabel('Sales ($)')
axes[0,1].set_ylabel('Frequency')
axes[0,1].set_yscale('log')

# Box plot
axes[1,0].boxplot(positive_sales, vert=True)
axes[1,0].set_title('Sales Box Plot (Sales > 0)')
axes[1,0].set_ylabel('Sales ($)')

# Sales by price ranges (excluding zero)
price_ranges = pd.cut(positive_sales, 
                     bins=[0, 1, 5, 10, 25, 50, 100, float('inf')],
                     labels=['$0-1', '$1-5', '$5-10', '$10-25', '$25-50', '$50-100', '$100+'])
range_counts = price_ranges.value_counts().sort_index()
axes[1,1].bar(range_counts.index, range_counts.values)
axes[1,1].set_title('Sales by Price Ranges (Sales > 0)')
axes[1,1].set_xlabel('Price Range')
axes[1,1].set_ylabel('Number of Sales')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Price range analysis
print("\nSales by price ranges (Sales > 0):")
for range_label, count in range_counts.items():
    percentage = count / len(positive_sales) * 100
    print(f"{range_label}: {count:,} sales ({percentage:.1f}%)")

## 3. Time Series Analysis

In [ ]:
# Time series analysis

# Aggregate daily sales
daily_sales = train.groupby('date')['sales'].agg(['sum', 'mean', 'count']).reset_index()

# Plot time series
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Total daily sales
axes[0,0].plot(daily_sales.date, daily_sales['sum'])
axes[0,0].set_title('Total Daily Sales')
axes[0,0].tick_params(axis='x', rotation=45)

# Average daily sales
axes[0,1].plot(daily_sales.date, daily_sales['mean'])
axes[0,1].set_title('Average Daily Sales per Store-Product')
axes[0,1].tick_params(axis='x', rotation=45)

# Sales distribution
axes[1,0].hist(train.sales[train.sales > 0], bins=50, alpha=0.7)
axes[1,0].set_title('Sales Distribution (Positive Sales Only)')
axes[1,0].set_xlabel('Sales')
axes[1,0].set_yscale('log')

# Monthly sales trend
monthly_sales = train.groupby(train.date.dt.to_period('M'))['sales'].sum()
axes[1,1].plot(monthly_sales.index.astype(str), monthly_sales.values)
axes[1,1].set_title('Monthly Sales Trend')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"Sales trend: {monthly_sales.iloc[-1]/monthly_sales.iloc[0]:.2f}x growth from start to end")

## 4. Analiza sprzedaży według krajów

In [ ]:
# Store analysis
store_sales = train.groupby('store_nbr')['sales'].agg(['sum', 'mean', 'count'])
store_info = store_sales.merge(stores, on='store_nbr')

# Top performing stores
print("Top 10 stores by total sales:")
print(store_info.nlargest(10, 'sum')[['sum', 'mean', 'city', 'state', 'type', 'cluster']])

# Store performance by attributes
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Sales by store type
type_sales = store_info.groupby('type')['sum'].mean()
axes[0,0].bar(type_sales.index, type_sales.values)
axes[0,0].set_title('Average Sales by Store Type')
axes[0,0].tick_params(axis='x', rotation=45)

# Sales by state
state_sales = store_info.groupby('state')['sum'].mean().sort_values(ascending=False).head(10)
axes[0,1].barh(state_sales.index, state_sales.values)
axes[0,1].set_title('Top 10 States by Average Store Sales')

# Store cluster analysis
cluster_sales = store_info.groupby('cluster')['sum'].mean()
axes[1,0].bar(cluster_sales.index, cluster_sales.values)
axes[1,0].set_title('Average Sales by Store Cluster')

# Store count by type
type_counts = stores['type'].value_counts()
axes[1,1].pie(type_counts.values, labels=type_counts.index, autopct='%1.1f%%')
axes[1,1].set_title('Store Distribution by Type')

plt.tight_layout()
plt.show()

print(f"\nStore performance varies {store_sales['sum'].max()/store_sales['sum'].min():.1f}x between best and worst")

## 5. Analiza produktów i kategorii

In [ ]:
# Product family analysis
family_sales = train.groupby('family')['sales'].agg(['sum', 'mean', 'count']).sort_values('sum', ascending=False)

print("Top product families by total sales:")
print(family_sales.head(10))

# Plot top families
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Top 15 families by total sales
top_families = family_sales.head(15)
axes[0,0].barh(range(len(top_families)), top_families['sum'])
axes[0,0].set_yticks(range(len(top_families)))
axes[0,0].set_yticklabels(top_families.index)
axes[0,0].set_title('Top 15 Product Families by Total Sales')

# Sales volatility (coefficient of variation)
family_cv = train.groupby('family')['sales'].apply(lambda x: x.std() / x.mean()).sort_values(ascending=False)
axes[0,1].bar(range(len(family_cv.head(10))), family_cv.head(10))
axes[0,1].set_xticks(range(len(family_cv.head(10))))
axes[0,1].set_xticklabels(family_cv.head(10).index, rotation=45, ha='right')
axes[0,1].set_title('Most Volatile Product Families (CV)')

# Family performance trend (recent vs early period)
early_period = train[train.date < '2015-01-01'].groupby('family')['sales'].sum()
recent_period = train[train.date >= '2016-01-01'].groupby('family')['sales'].sum()
growth_rate = (recent_period / early_period).sort_values(ascending=False).head(10)

axes[1,0].bar(range(len(growth_rate)), growth_rate)
axes[1,0].set_xticks(range(len(growth_rate)))
axes[1,0].set_xticklabels(growth_rate.index, rotation=45, ha='right')
axes[1,0].set_title('Fastest Growing Product Families')
axes[1,0].axhline(y=1, color='red', linestyle='--', alpha=0.7)

# Sales seasonality for top family
top_family = family_sales.index[0]
top_family_data = train[train.family == top_family]
monthly_pattern = top_family_data.groupby(top_family_data.date.dt.month)['sales'].mean()
axes[1,1].plot(monthly_pattern.index, monthly_pattern.values, marker='o')
axes[1,1].set_title(f'Monthly Seasonality - {top_family}')
axes[1,1].set_xlabel('Month')
axes[1,1].set_xticks(range(1, 13))

plt.tight_layout()
plt.show()

## 6. Individual Product Analysis

In [ ]:
# Individual product analysis - detailed look at specific products

# Select interesting products for detailed analysis
top_products = []
for family in family_sales.head(5).index:
    # Get representative store for each top family
    family_data = train[train.family == family]
    top_store = family_data.groupby('store_nbr')['sales'].sum().idxmax()
    top_products.append((family, top_store))

print("Selected products for detailed analysis:")
for family, store in top_products:
    print(f"- {family} at Store {store}")

# Create detailed analysis for each selected product
fig, axes = plt.subplots(len(top_products), 3, figsize=(20, 5*len(top_products)))
if len(top_products) == 1:
    axes = axes.reshape(1, -1)

for i, (family, store) in enumerate(top_products):
    product_data = train[(train.family == family) & (train.store_nbr == store)].copy()
    product_data = product_data.sort_values('date')
    
    # 1. Time series with trend line
    axes[i,0].plot(product_data.date, product_data.sales, alpha=0.7, linewidth=1)
    
    # Add trend line
    x_numeric = np.arange(len(product_data))
    z = np.polyfit(x_numeric, product_data.sales, 1)
    p = np.poly1d(z)
    axes[i,0].plot(product_data.date, p(x_numeric), "r--", alpha=0.8, linewidth=2)
    
    axes[i,0].set_title(f'{family} (Store {store}) - Sales Trend')
    axes[i,0].tick_params(axis='x', rotation=45)
    axes[i,0].set_ylabel('Sales')
    
    # Calculate trend statistics
    trend_slope = z[0]
    avg_sales = product_data.sales.mean()
    trend_pct = (trend_slope * 365) / avg_sales * 100  # Annual trend percentage
    
    # 2. Seasonal patterns (monthly)
    monthly_avg = product_data.groupby(product_data.date.dt.month)['sales'].mean()
    axes[i,1].bar(monthly_avg.index, monthly_avg.values, alpha=0.7)
    axes[i,1].set_title(f'{family} - Monthly Seasonality\n(Trend: {trend_pct:+.1f}% per year)')
    axes[i,1].set_xlabel('Month')
    axes[i,1].set_ylabel('Average Sales')
    axes[i,1].set_xticks(range(1, 13))
    
    # 3. Weekly patterns
    product_data['weekday'] = product_data.date.dt.dayofweek
    weekly_avg = product_data.groupby('weekday')['sales'].mean()
    weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    axes[i,2].bar(range(7), weekly_avg.values, alpha=0.7)
    axes[i,2].set_title(f'{family} - Weekly Patterns')
    axes[i,2].set_xlabel('Day of Week')
    axes[i,2].set_ylabel('Average Sales')
    axes[i,2].set_xticks(range(7))
    axes[i,2].set_xticklabels(weekday_names)

plt.tight_layout()
plt.show()

# Product correlation analysis
print("\n=== PRODUCT CORRELATION ANALYSIS ===")

# Create a matrix of top families sales by date
top_families_list = family_sales.head(8).index.tolist()
family_pivot = train[train.family.isin(top_families_list)].groupby(['date', 'family'])['sales'].sum().unstack(fill_value=0)

# Calculate correlation matrix
corr_matrix = family_pivot.corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
plt.title('Product Family Sales Correlation Matrix')
plt.tight_layout()
plt.show()

# Find most and least correlated product pairs
corr_values = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        family1 = corr_matrix.columns[i]
        family2 = corr_matrix.columns[j]
        corr_val = corr_matrix.iloc[i, j]
        corr_values.append((family1, family2, corr_val))

corr_df = pd.DataFrame(corr_values, columns=['Family1', 'Family2', 'Correlation'])
corr_df = corr_df.sort_values('Correlation', ascending=False)

print("\nMost correlated product families:")
print(corr_df.head(5)[['Family1', 'Family2', 'Correlation']])

print("\nLeast correlated product families:")
print(corr_df.tail(5)[['Family1', 'Family2', 'Correlation']])

# Promotion impact analysis for individual products
print("\n=== PROMOTION IMPACT ANALYSIS ===")

# Check if we have promotion data in the dataset
if 'onpromotion' in train.columns:
    print("Analyzing promotion impact on sales...")
    
    promotion_analysis = []
    for family in top_families_list[:5]:  # Analyze top 5 families
        family_data = train[train.family == family].copy()
        
        # Sales with and without promotion
        promo_sales = family_data[family_data.onpromotion > 0]['sales']
        no_promo_sales = family_data[family_data.onpromotion == 0]['sales']
        
        if len(promo_sales) > 0 and len(no_promo_sales) > 0:
            promo_avg = promo_sales.mean()
            no_promo_avg = no_promo_sales.mean()
            lift = (promo_avg - no_promo_avg) / no_promo_avg * 100
            
            promotion_analysis.append({
                'Family': family,
                'Avg_Sales_No_Promo': no_promo_avg,
                'Avg_Sales_Promo': promo_avg,
                'Promotion_Lift_%': lift,
                'Promo_Records': len(promo_sales)
            })
    
    if promotion_analysis:
        promo_df = pd.DataFrame(promotion_analysis)
        promo_df = promo_df.sort_values('Promotion_Lift_%', ascending=False)
        
        print("\nPromotion effectiveness by product family:")
        print(promo_df.round(2))
        
        # Plot promotion lift
        plt.figure(figsize=(12, 6))
        bars = plt.bar(range(len(promo_df)), promo_df['Promotion_Lift_%'], alpha=0.7)
        plt.title('Promotion Lift by Product Family')
        plt.xlabel('Product Family')
        plt.ylabel('Sales Lift (%)')
        plt.xticks(range(len(promo_df)), promo_df['Family'], rotation=45, ha='right')
        plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
        
        # Add value labels on bars
        for i, bar in enumerate(bars):
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2., height + (height*0.01 if height > 0 else height*0.01),
                    f'{height:.1f}%', ha='center', va='bottom' if height > 0 else 'top')
        
        plt.tight_layout()
        plt.show()
    else:
        print("No promotion data found for analysis.")
else:
    print("No 'onpromotion' column found in the dataset.")
    
    # Alternative: analyze sales spikes as potential promotions
    print("\nAnalyzing sales spikes as potential promotional periods...")
    
    for family in top_families_list[:3]:
        family_data = train[train.family == family].copy()
        family_daily = family_data.groupby('date')['sales'].sum().reset_index()
        
        # Define spike as sales > 95th percentile
        threshold = family_daily['sales'].quantile(0.95)
        spike_days = family_daily[family_daily['sales'] > threshold]
        normal_days = family_daily[family_daily['sales'] <= threshold]
        
        if len(spike_days) > 0:
            spike_avg = spike_days['sales'].mean()
            normal_avg = normal_days['sales'].mean()
            spike_lift = (spike_avg - normal_avg) / normal_avg * 100
            
            print(f"\n{family}:")
            print(f"  Normal days avg sales: ${normal_avg:,.2f}")
            print(f"  Spike days avg sales: ${spike_avg:,.2f}")
            print(f"  Spike lift: {spike_lift:.1f}%")
            print(f"  Spike days count: {len(spike_days)} ({len(spike_days)/len(family_daily)*100:.1f}% of days)")

## 7. External Factors Analysis

In [ ]:
# External factors analysis
print("=== EXTERNAL FACTORS ANALYSIS ===")

# First, let's check what external data files are available
import os
data_dir = '../data/raw'
if os.path.exists(data_dir):
    data_files = [f for f in os.listdir(data_dir) if f.endswith('.csv')]
    print(f"Available data files: {data_files}")
else:
    print("Data directory not found. Assuming files are in current directory.")
    data_files = [f for f in os.listdir('.') if f.endswith('.csv')]
    print(f"CSV files in current directory: {data_files}")

# Try to load external data files
external_data = {}
file_patterns = {
    'oil': ['oil.csv', 'oil_prices.csv'],
    'holidays': ['holidays_events.csv', 'holidays.csv'],
    'stores': ['stores.csv', 'store_info.csv'],
    'transactions': ['transactions.csv']
}

for data_type, patterns in file_patterns.items():
    for pattern in patterns:
        try:
            if pattern in data_files:
                if data_dir and os.path.exists(os.path.join(data_dir, pattern)):
                    external_data[data_type] = pd.read_csv(os.path.join(data_dir, pattern))
                else:
                    external_data[data_type] = pd.read_csv(pattern)
                print(f"Loaded {data_type} data from {pattern}")
                break
        except Exception as e:
            print(f"Could not load {pattern}: {e}")

print(f"\nLoaded external datasets: {list(external_data.keys())}")

# Analyze each external factor
if external_data:
    
    # Oil prices analysis
    if 'oil' in external_data:
        print("\n=== OIL PRICES ANALYSIS ===")
        oil_data = external_data['oil'].copy()
        print(f"Oil data shape: {oil_data.shape}")
        print(f"Oil data columns: {oil_data.columns.tolist()}")
        
        if 'date' in oil_data.columns:
            oil_data['date'] = pd.to_datetime(oil_data['date'])
            
            # Basic oil price statistics
            price_col = [col for col in oil_data.columns if 'price' in col.lower() or 'oil' in col.lower()]
            if price_col:
                price_col = price_col[0]
                print(f"\nOil price statistics ({price_col}):")
                print(oil_data[price_col].describe())
                
                # Plot oil prices over time
                plt.figure(figsize=(15, 6))
                plt.subplot(1, 2, 1)
                plt.plot(oil_data['date'], oil_data[price_col], linewidth=1.5)
                plt.title('Oil Prices Over Time')
                plt.xlabel('Date')
                plt.ylabel('Oil Price')
                plt.xticks(rotation=45)
                
                # Oil price correlation with total sales
                # Aggregate sales by date
                daily_sales = train.groupby('date')['sales'].sum().reset_index()
                daily_sales['date'] = pd.to_datetime(daily_sales['date'])
                
                # Merge with oil data
                sales_oil = daily_sales.merge(oil_data, on='date', how='inner')
                
                if len(sales_oil) > 0:
                    correlation = sales_oil['sales'].corr(sales_oil[price_col])
                    print(f"\nCorrelation between oil prices and total sales: {correlation:.3f}")
                    
                    plt.subplot(1, 2, 2)
                    plt.scatter(sales_oil[price_col], sales_oil['sales'], alpha=0.6)
                    plt.xlabel('Oil Price')
                    plt.ylabel('Total Daily Sales')
                    plt.title(f'Sales vs Oil Price\n(Correlation: {correlation:.3f})')
                    
                    # Add trend line
                    z = np.polyfit(sales_oil[price_col], sales_oil['sales'], 1)
                    p = np.poly1d(z)
                    plt.plot(sales_oil[price_col], p(sales_oil[price_col]), "r--", alpha=0.8)
                
                plt.tight_layout()
                plt.show()
    
    # Holidays analysis
    if 'holidays' in external_data:
        print("\n=== HOLIDAYS ANALYSIS ===")
        holidays_data = external_data['holidays'].copy()
        print(f"Holidays data shape: {holidays_data.shape}")
        print(f"Holidays data columns: {holidays_data.columns.tolist()}")
        
        if 'date' in holidays_data.columns:
            holidays_data['date'] = pd.to_datetime(holidays_data['date'])
            
            # Count holidays by type
            if 'type' in holidays_data.columns:
                holiday_types = holidays_data['type'].value_counts()
                print(f"\nHoliday types:")
                print(holiday_types)
                
                plt.figure(figsize=(12, 8))
                
                plt.subplot(2, 2, 1)
                holiday_types.plot(kind='bar', alpha=0.7)
                plt.title('Holiday Types Distribution')
                plt.xticks(rotation=45)
                
            # Analyze sales impact of holidays
            daily_sales = train.groupby('date')['sales'].sum().reset_index()
            daily_sales['date'] = pd.to_datetime(daily_sales['date'])
            
            # Mark holiday dates
            holiday_dates = set(holidays_data['date'].dt.date)
            daily_sales['is_holiday'] = daily_sales['date'].dt.date.isin(holiday_dates)
            
            holiday_sales = daily_sales[daily_sales['is_holiday']]['sales']
            normal_sales = daily_sales[~daily_sales['is_holiday']]['sales']
            
            print(f"\nSales comparison:")
            print(f"Average sales on holidays: ${holiday_sales.mean():,.2f}")
            print(f"Average sales on normal days: ${normal_sales.mean():,.2f}")
            
            if len(holiday_sales) > 0 and len(normal_sales) > 0:
                holiday_lift = (holiday_sales.mean() - normal_sales.mean()) / normal_sales.mean() * 100
                print(f"Holiday sales lift: {holiday_lift:+.1f}%")
                
                plt.subplot(2, 2, 2)
                plt.boxplot([normal_sales, holiday_sales], labels=['Normal Days', 'Holidays'])
                plt.title(f'Sales Distribution\n(Holiday Lift: {holiday_lift:+.1f}%)')
                plt.ylabel('Daily Sales')
                
            # Holiday calendar heatmap
            if len(daily_sales) > 0:
                daily_sales['year'] = daily_sales['date'].dt.year
                daily_sales['month'] = daily_sales['date'].dt.month
                daily_sales['day'] = daily_sales['date'].dt.day
                
                # Create monthly holiday counts
                holidays_data['year'] = holidays_data['date'].dt.year
                holidays_data['month'] = holidays_data['date'].dt.month
                monthly_holidays = holidays_data.groupby(['year', 'month']).size().reset_index(name='holiday_count')
                
                # Sample for visualization (latest year)
                if len(monthly_holidays) > 0:
                    latest_year = monthly_holidays['year'].max()
                    year_holidays = monthly_holidays[monthly_holidays['year'] == latest_year]
                    
                    plt.subplot(2, 2, 3)
                    if len(year_holidays) > 0:
                        plt.bar(year_holidays['month'], year_holidays['holiday_count'], alpha=0.7)
                        plt.title(f'Monthly Holidays Count ({latest_year})')
                        plt.xlabel('Month')
                        plt.ylabel('Number of Holidays')
                        plt.xticks(range(1, 13))
                
            plt.tight_layout()
            plt.show()
    
    # Transactions analysis
    if 'transactions' in external_data:
        print("\n=== TRANSACTIONS ANALYSIS ===")
        trans_data = external_data['transactions'].copy()
        print(f"Transactions data shape: {trans_data.shape}")
        print(f"Transactions data columns: {trans_data.columns.tolist()}")
        
        if 'date' in trans_data.columns:
            trans_data['date'] = pd.to_datetime(trans_data['date'])
            
            # Transactions statistics
            trans_col = [col for col in trans_data.columns if 'transaction' in col.lower()][0]
            print(f"\nTransactions statistics:")
            print(trans_data[trans_col].describe())
            
            # Merge with sales data for correlation
            daily_sales = train.groupby(['date', 'store_nbr'])['sales'].sum().reset_index()
            daily_sales['date'] = pd.to_datetime(daily_sales['date'])
            
            sales_trans = daily_sales.merge(trans_data, on=['date', 'store_nbr'], how='inner')
            
            if len(sales_trans) > 0:
                correlation = sales_trans['sales'].corr(sales_trans[trans_col])
                print(f"\nCorrelation between transactions and sales: {correlation:.3f}")
                
                plt.figure(figsize=(15, 5))
                
                # Time series comparison
                plt.subplot(1, 3, 1)
                monthly_sales = sales_trans.groupby(sales_trans['date'].dt.to_period('M'))['sales'].sum()
                monthly_trans = sales_trans.groupby(sales_trans['date'].dt.to_period('M'))[trans_col].sum()
                
                ax1 = plt.gca()
                ax2 = ax1.twinx()
                
                line1 = ax1.plot(monthly_sales.index.astype(str), monthly_sales.values, 'b-', alpha=0.7, label='Sales')
                line2 = ax2.plot(monthly_trans.index.astype(str), monthly_trans.values, 'r-', alpha=0.7, label='Transactions')
                
                ax1.set_xlabel('Month')
                ax1.set_ylabel('Sales', color='b')
                ax2.set_ylabel('Transactions', color='r')
                ax1.tick_params(axis='x', rotation=45)
                plt.title('Sales vs Transactions Over Time')
                
                # Scatter plot
                plt.subplot(1, 3, 2)
                plt.scatter(sales_trans[trans_col], sales_trans['sales'], alpha=0.5)
                plt.xlabel('Transactions')
                plt.ylabel('Sales')
                plt.title(f'Sales vs Transactions\n(Correlation: {correlation:.3f})')
                
                # Add trend line
                z = np.polyfit(sales_trans[trans_col], sales_trans['sales'], 1)
                p = np.poly1d(z)
                plt.plot(sales_trans[trans_col], p(sales_trans[trans_col]), "r--", alpha=0.8)
                
                # Average transaction value
                sales_trans['avg_transaction_value'] = sales_trans['sales'] / sales_trans[trans_col]
                sales_trans['avg_transaction_value'] = sales_trans['avg_transaction_value'].replace([np.inf, -np.inf], np.nan)
                
                plt.subplot(1, 3, 3)
                avg_trans_value = sales_trans.groupby('store_nbr')['avg_transaction_value'].mean().dropna()
                plt.hist(avg_trans_value, bins=20, alpha=0.7, edgecolor='black')
                plt.xlabel('Average Transaction Value')
                plt.ylabel('Number of Stores')
                plt.title('Distribution of Avg Transaction Value by Store')
                
                plt.tight_layout()
                plt.show()
                
                print(f"\nAverage transaction value statistics:")
                print(avg_trans_value.describe())
    
    # Store information analysis
    if 'stores' in external_data:
        print("\n=== STORE INFORMATION ANALYSIS ===")
        stores_data = external_data['stores'].copy()
        print(f"Stores data shape: {stores_data.shape}")
        print(f"Stores data columns: {stores_data.columns.tolist()}")
        
        # Merge store info with sales
        store_sales = train.groupby('store_nbr')['sales'].agg(['sum', 'mean', 'count']).reset_index()
        store_analysis = store_sales.merge(stores_data, left_on='store_nbr', right_on='store_nbr', how='left')
        
        # Analyze by store attributes
        for col in stores_data.columns:
            if col != 'store_nbr' and stores_data[col].dtype in ['object', 'category']:
                print(f"\nSales by {col}:")
                group_stats = store_analysis.groupby(col)['sum'].agg(['mean', 'count'])
                print(group_stats)
        
        # Visualize store performance by attributes
        categorical_cols = [col for col in stores_data.columns 
                          if col != 'store_nbr' and stores_data[col].dtype in ['object', 'category']]
        
        if categorical_cols:
            n_cols = min(3, len(categorical_cols))
            fig, axes = plt.subplots(1, n_cols, figsize=(6*n_cols, 6))
            if n_cols == 1:
                axes = [axes]
            
            for i, col in enumerate(categorical_cols[:n_cols]):
                group_sales = store_analysis.groupby(col)['sum'].mean()
                group_sales.plot(kind='bar', ax=axes[i], alpha=0.7)
                axes[i].set_title(f'Average Total Sales by {col}')
                axes[i].set_ylabel('Average Total Sales')
                axes[i].tick_params(axis='x', rotation=45)
            
            plt.tight_layout()
            plt.show()

else:
    print("No external data files found. Performing analysis on available train data only.")
    
    # Alternative analysis using patterns in the training data
    print("\n=== TEMPORAL PATTERNS ANALYSIS ===")
    
    # Day of week patterns
    train['weekday'] = train['date'].dt.dayofweek
    weekday_sales = train.groupby('weekday')['sales'].mean()
    
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    plt.bar(range(7), weekday_sales.values, alpha=0.7)
    plt.title('Average Sales by Day of Week')
    plt.xlabel('Day of Week')
    plt.ylabel('Average Sales')
    plt.xticks(range(7), weekday_names)
    
    # Monthly patterns
    monthly_sales = train.groupby(train['date'].dt.month)['sales'].mean()
    plt.subplot(1, 3, 2)
    plt.bar(monthly_sales.index, monthly_sales.values, alpha=0.7)
    plt.title('Average Sales by Month')
    plt.xlabel('Month')
    plt.ylabel('Average Sales')
    plt.xticks(range(1, 13))
    
    # Year-over-year growth
    yearly_sales = train.groupby(train['date'].dt.year)['sales'].sum()
    plt.subplot(1, 3, 3)
    yearly_growth = yearly_sales.pct_change() * 100
    plt.bar(yearly_growth.index[1:], yearly_growth.values[1:], alpha=0.7)
    plt.title('Year-over-Year Sales Growth')
    plt.xlabel('Year')
    plt.ylabel('Growth (%)')
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()
    
    print("\nWeekday sales patterns:")
    for i, day in enumerate(weekday_names):
        print(f"{day}: ${weekday_sales.iloc[i]:,.2f}")
    
    print("\nYearly sales totals:")
    for year, total in yearly_sales.items():
        print(f"{year}: ${total:,.2f}")

print("\n=== DATA EXPLORATION SUMMARY ===")
print("\nKey Findings:")
print("1. Dataset contains sales data for 54 stores and 33 product families")
print(f"2. Time period: {train['date'].min()} to {train['date'].max()}")
print(f"3. Total sales records: {len(train):,}")
print(f"4. Zero sales records: {(train['sales'] == 0).sum():,} ({(train['sales'] == 0).mean()*100:.1f}%)")
print(f"5. Average daily sales per store-product: ${train['sales'].mean():.2f}")
print(f"6. Total sales value: ${train['sales'].sum():,.2f}")
print("\nNext steps: Data preprocessing, feature engineering, and model development")

## 8. Conclusion

This comprehensive data exploration has revealed several key insights about the Store Sales Time Series dataset:

### Key Findings:
1. **Data Structure**: Complete grid format with 54 stores × 33 product families × 1,684 days
2. **Data Quality**: ~31% zero sales records, indicating either no stock or no demand
3. **Sales Distribution**: Wide range from $0.01 to $3,502, with most sales under $20
4. **Top Performers**: GROCERY I, BEVERAGES, and PRODUCE are the highest-selling families
5. **Store Patterns**: Significant variation in performance across stores and locations
6. **Seasonality**: Clear monthly and weekly patterns in sales data
7. **Growth Trends**: Generally positive trends for most product families
8. **External Factors**: Oil prices, holidays, and transaction counts show varying correlations with sales

### Implications for Modeling:
- Need to handle zero sales appropriately (separate model or special treatment)
- Store and product family clustering could improve predictions
- Seasonal patterns should be incorporated as features
- External factors like holidays and promotions are important
- Individual product analysis reveals unique patterns that may require family-specific models

### Next Steps:
1. **Data Preprocessing**: Handle zeros, outliers, and missing values
2. **Feature Engineering**: Create lag features, rolling statistics, seasonal features
3. **Baseline Models**: Start with simple approaches (moving averages, linear regression)
4. **Advanced Models**: Implement neural networks (LSTM, CNN, Transformer-based)
5. **Model Evaluation**: Use appropriate time series validation techniques

In [ ]:
# Individual product family deep dive
print("=== INDIVIDUAL PRODUCT FAMILY ANALYSIS ===")

# Select top 6 families for detailed analysis
top_6_families = family_sales.head(6).index.tolist()

fig, axes = plt.subplots(3, 2, figsize=(16, 12))
axes = axes.flatten()

for i, family in enumerate(top_6_families):
    family_data = train[train.family == family]
    
    # Daily sales for this family
    daily_family_sales = family_data.groupby('date')['sales'].sum()
    
    axes[i].plot(daily_family_sales.index, daily_family_sales.values, alpha=0.7)
    axes[i].set_title(f'{family}\nTotal Sales: ${family_sales.loc[family, "sum"]:,.0f}')
    axes[i].tick_params(axis='x', rotation=45)
    axes[i].grid(True, alpha=0.3)
    
    # Add trend line
    z = np.polyfit(range(len(daily_family_sales)), daily_family_sales.values, 1)
    p = np.poly1d(z)
    axes[i].plot(daily_family_sales.index, p(range(len(daily_family_sales))), 
                "r--", alpha=0.8, linewidth=2)

plt.tight_layout()
plt.show()

# Seasonality comparison for top families
print("\n=== SEASONAL PATTERNS COMPARISON ===")

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for i, family in enumerate(top_6_families):
    family_data = train[train.family == family]
    
    # Monthly seasonality
    monthly_pattern = family_data.groupby(family_data.date.dt.month)['sales'].mean()
    
    axes[i].plot(monthly_pattern.index, monthly_pattern.values, marker='o', linewidth=2, markersize=8)
    axes[i].set_title(f'{family} - Monthly Pattern')
    axes[i].set_xlabel('Month')
    axes[i].set_ylabel('Average Sales')
    axes[i].set_xticks(range(1, 13))
    axes[i].grid(True, alpha=0.3)
    
    # Highlight peak months
    peak_month = monthly_pattern.idxmax()
    axes[i].scatter(peak_month, monthly_pattern[peak_month], 
                   color='red', s=100, zorder=5)
    axes[i].annotate(f'Peak: {peak_month}', 
                    xy=(peak_month, monthly_pattern[peak_month]),
                    xytext=(peak_month+1, monthly_pattern[peak_month]*1.1),
                    arrowprops=dict(arrowstyle='->', color='red'))

plt.tight_layout()
plt.show()

# Weekly patterns for top families
print("\n=== WEEKLY PATTERNS COMPARISON ===")

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

for i, family in enumerate(top_6_families):
    family_data = train[train.family == family]
    
    # Weekly pattern (0=Monday, 6=Sunday)
    weekly_pattern = family_data.groupby(family_data.date.dt.dayofweek)['sales'].mean()
    
    bars = axes[i].bar(range(7), weekly_pattern.values, alpha=0.7)
    axes[i].set_title(f'{family} - Weekly Pattern')
    axes[i].set_xlabel('Day of Week')
    axes[i].set_ylabel('Average Sales')
    axes[i].set_xticks(range(7))
    axes[i].set_xticklabels([day[:3] for day in days_of_week], rotation=45)
    axes[i].grid(True, alpha=0.3, axis='y')
    
    # Highlight weekend days
    bars[5].set_color('orange')  # Saturday
    bars[6].set_color('red')     # Sunday

plt.tight_layout()
plt.show()

# Product family statistics summary
print("\n=== TOP FAMILIES SUMMARY STATISTICS ===")
for family in top_6_families:
    family_data = train[train.family == family]
    total_sales = family_data['sales'].sum()
    avg_daily = family_data.groupby('date')['sales'].sum().mean()
    zero_days = (family_data.groupby('date')['sales'].sum() == 0).sum()
    peak_day = family_data.groupby('date')['sales'].sum().max()
    
    print(f"\n{family}:")
    print(f"  Total Sales: ${total_sales:,.0f}")
    print(f"  Avg Daily Sales: ${avg_daily:.0f}")
    print(f"  Days with Zero Sales: {zero_days}")
    print(f"  Peak Daily Sales: ${peak_day:.0f}")
    print(f"  Sales Volatility (CV): {family_cv[family]:.2f}")

In [ ]:
# Product family correlation analysis
print("\n=== PRODUCT FAMILY CORRELATION ANALYSIS ===")

# Create daily sales matrix for families
family_daily_sales = train.groupby(['date', 'family'])['sales'].sum().unstack(fill_value=0)

# Calculate correlation matrix for top 10 families
top_10_families = family_sales.head(10).index
corr_matrix = family_daily_sales[top_10_families].corr()

# Plot correlation heatmap
fig, ax = plt.subplots(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f', cbar_kws={'shrink': 0.8})
ax.set_title('Product Family Sales Correlation Matrix (Top 10)', fontsize=14, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Find most and least correlated pairs
corr_pairs = []
for i in range(len(top_10_families)):
    for j in range(i+1, len(top_10_families)):
        family1, family2 = top_10_families[i], top_10_families[j]
        corr_val = corr_matrix.loc[family1, family2]
        corr_pairs.append((family1, family2, corr_val))

corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)

print("\nMost correlated product families:")
for family1, family2, corr in corr_pairs[:5]:
    print(f"{family1} & {family2}: {corr:.3f}")

print("\nLeast correlated product families:")
for family1, family2, corr in corr_pairs[-5:]:
    print(f"{family1} & {family2}: {corr:.3f}")

In [ ]:
# Promotion impact analysis by product family
print("\n=== PROMOTION IMPACT ANALYSIS ===")

# Calculate promotion statistics for top families
promotion_stats = []
for family in top_6_families:
    family_data = train[train.family == family]
    
    # Sales with and without promotion
    promo_sales = family_data[family_data.onpromotion == 1]['sales']
    no_promo_sales = family_data[family_data.onpromotion == 0]['sales']
    
    # Remove zeros for better comparison
    promo_sales_pos = promo_sales[promo_sales > 0]
    no_promo_sales_pos = no_promo_sales[no_promo_sales > 0]
    
    if len(promo_sales_pos) > 0 and len(no_promo_sales_pos) > 0:
        promo_mean = promo_sales_pos.mean()
        no_promo_mean = no_promo_sales_pos.mean()
        lift = (promo_mean / no_promo_mean - 1) * 100
        
        promotion_stats.append({
            'family': family,
            'promo_mean': promo_mean,
            'no_promo_mean': no_promo_mean,
            'lift_percent': lift,
            'promo_records': len(promo_sales),
            'total_records': len(family_data)
        })

# Create promotion impact visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Promotion lift chart
families = [stat['family'] for stat in promotion_stats]
lifts = [stat['lift_percent'] for stat in promotion_stats]

bars = axes[0].barh(families, lifts, color=['green' if x > 0 else 'red' for x in lifts])
axes[0].set_title('Promotion Lift by Product Family (%)')
axes[0].set_xlabel('Sales Lift (%)')
axes[0].axvline(x=0, color='black', linestyle='-', alpha=0.3)
axes[0].grid(True, alpha=0.3)

# Add value labels on bars
for i, (bar, lift) in enumerate(zip(bars, lifts)):
    axes[0].text(lift + (5 if lift > 0 else -5), bar.get_y() + bar.get_height()/2, 
                f'{lift:.1f}%', va='center', ha='left' if lift > 0 else 'right')

# Promotion frequency chart
promo_freq = [stat['promo_records'] / stat['total_records'] * 100 for stat in promotion_stats]

bars2 = axes[1].barh(families, promo_freq, alpha=0.7, color='blue')
axes[1].set_title('Promotion Frequency by Product Family (%)')
axes[1].set_xlabel('% of Records on Promotion')
axes[1].grid(True, alpha=0.3)

# Add value labels
for bar, freq in zip(bars2, promo_freq):
    axes[1].text(freq + 1, bar.get_y() + bar.get_height()/2, 
                f'{freq:.1f}%', va='center', ha='left')

plt.tight_layout()
plt.show()

# Print promotion statistics
print("\nPromotion impact summary:")
for stat in promotion_stats:
    print(f"\n{stat['family']}:")
    print(f"  Average sales (no promo): ${stat['no_promo_mean']:.2f}")
    print(f"  Average sales (with promo): ${stat['promo_mean']:.2f}")
    print(f"  Promotion lift: {stat['lift_percent']:.1f}%")
    print(f"  Promotion frequency: {stat['promo_records']/stat['total_records']*100:.1f}%")

### Individual Product Analysis Summary:

**Key Insights from Product-Level Analysis:**

1. **Sales Trends**: Each product family shows distinct growth patterns and seasonal behaviors
2. **Seasonality**: Clear monthly and weekly patterns vary significantly between product categories
3. **Correlations**: Some product families are highly correlated (substitute/complementary products)
4. **Promotion Impact**: Promotional effectiveness varies dramatically across product families
5. **Volatility**: Different product categories show varying levels of sales predictability

**Business Implications:**
- Product-specific forecasting models may be necessary
- Promotional strategies should be tailored by product family
- Cross-selling opportunities exist for highly correlated products
- Seasonal inventory planning needs to account for family-specific patterns

## 6. External Factors Analysis

In [ ]:
# Oil price analysis
print(f"Oil data coverage: {oil.date.min()} to {oil.date.max()}")
print(f"Missing oil price days: {oil.dcoilwtico.isnull().sum()}")

# Merge with daily sales for correlation
daily_oil = daily_sales.merge(oil, on='date', how='left')
correlation = daily_oil[['sum', 'dcoilwtico']].corr().iloc[0,1]
print(f"Correlation between oil prices and total sales: {correlation:.3f}")

# Holiday analysis
print(f"\nHoliday types: {holidays.type.value_counts()}")
print(f"Holiday impact on national level: {holidays.locale.value_counts()}")

# Plot external factors
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Oil prices over time
axes[0,0].plot(oil.date, oil.dcoilwtico)
axes[0,0].set_title('Oil Prices Over Time')
axes[0,0].tick_params(axis='x', rotation=45)

# Sales vs oil prices scatter
valid_data = daily_oil.dropna()
axes[0,1].scatter(valid_data.dcoilwtico, valid_data['sum'], alpha=0.6)
axes[0,1].set_xlabel('Oil Price')
axes[0,1].set_ylabel('Total Daily Sales')
axes[0,1].set_title(f'Sales vs Oil Price (r={correlation:.3f})')

# Holiday frequency by month
holiday_monthly = holidays.groupby(holidays.date.dt.month).size()
axes[1,0].bar(holiday_monthly.index, holiday_monthly.values)
axes[1,0].set_title('Holidays by Month')
axes[1,0].set_xlabel('Month')
axes[1,0].set_xticks(range(1, 13))

# Transaction pattern
if not transactions.empty:
    daily_transactions = transactions.groupby('date')['transactions'].sum()
    axes[1,1].plot(daily_transactions.index, daily_transactions.values)
    axes[1,1].set_title('Daily Transaction Volume')
    axes[1,1].tick_params(axis='x', rotation=45)
else:
    axes[1,1].text(0.5, 0.5, 'No transaction data', ha='center', va='center')
    axes[1,1].set_title('Transaction Data Not Available')

plt.tight_layout()
plt.show()

## 6. Wnioski z analizy eksploracyjnej

### Key Findings:

**Data Quality:**
- Dataset spans multiple years with consistent daily records
- Oil price data has some missing values
- First day of a year always has 0 sales 

**Sales Patterns:**
- Clear growth trend over time
- Seasonal patterns visible in monthly aggregations
- High variability between stores and product families

**Store Performance:**
- Significant performance differences across store types and locations
- Store clusters show distinct sales patterns
- Geographic concentration affects performance

**Product Insights:**
- Top product families dominate total sales
- Different families show varying seasonality and growth rates
- Some categories are more volatile than others

**External Factors:**
- Oil prices show some correlation with sales (higher the price -> lower the sales)
- Holidays concentrated in certain months
- Transaction volume tracks with sales patterns

### Modeling Implications:
- Consider store-specific models or clustering
- Include external factors (oil prices, holidays)
- Account for seasonality and trends
- Handle negative sales appropriately
- Feature engineering for location and store characteristics